In [3]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np

from category_encoders.target_encoder import TargetEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

In [4]:
train = pd.read_csv('train_base.csv', sep=';')
test = pd.read_csv('test_base.csv', sep=';') 

In [5]:
from analytics import Analysis as als

In [6]:
bureau = pd.read_csv('bureau.csv')

In [7]:
list(bureau)

['SK_ID_CURR',
 'SK_ID_BUREAU',
 'CREDIT_ACTIVE',
 'CREDIT_CURRENCY',
 'DAYS_CREDIT',
 'CREDIT_DAY_OVERDUE',
 'DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE',
 'CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_SUM_OVERDUE',
 'CREDIT_TYPE',
 'DAYS_CREDIT_UPDATE',
 'AMT_ANNUITY']

In [8]:
bureau.shape

(1716428, 17)

In [9]:
als.check_missing_values(bureau, 10, 0)

AMT_ANNUITY               1226791
AMT_CREDIT_MAX_OVERDUE    1124488
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_DEBT        257669
DAYS_CREDIT_ENDDATE        105553
AMT_CREDIT_SUM                 13
CREDIT_TYPE                     0
AMT_CREDIT_SUM_OVERDUE          0
CNT_CREDIT_PROLONG              0
dtype: int64

In [10]:
als.check_missing_values(bureau, 15, 1)

793046     6
773821     6
871866     6
468959     6
701505     6
871863     6
1291779    6
552590     6
311889     6
1315567    6
1174161    6
871824     6
871823     6
311862     6
1011092    6
dtype: int64

In [11]:
als.get_unique_value(bureau, None)

SK_ID_CURR <class 'numpy.int64'> [215354 162297 402440 ... 448157 345866 235871]
SK_ID_BUREAU <class 'numpy.int64'> [5714462 5714463 5714464 ... 5057762 5057770 5057778]
CREDIT_ACTIVE <class 'str'> ['Closed' 'Active' 'Sold' 'Bad debt']
CREDIT_CURRENCY <class 'str'> ['currency 1' 'currency 2' 'currency 4' 'currency 3']
DAYS_CREDIT <class 'numpy.int64'> [-497 -208 -203 ...    0   -3   -1]
CREDIT_DAY_OVERDUE <class 'numpy.int64'> [   0 2603    6   30 2156  496  186 2264   41    8   71   13  252 1201
   12   38 1777  150   18   37   64 2050    9    4   29   22   49 1930
   34  951  936  664   45 2625  837   21   15   31 1715   19   67    7
   26   23   48   46   73   90 1127   53   33 1572   39 2582   14   25
   10   50   17  193   40   20   28  123  138  700  122 1504   32  670
   75  140   24  277 2019  161  962  187   58  106  634   83  120   60
   93 1694  227   44  344 2085 1248  124 1651 2541 1541  642  745  163
  234 1472  688 2455  614  319 2187   68  831  139  283  180   47  164
 

# Анализ данных

## Категориальные признаки:
### Агрегирование:
#### Для агрегирования категориальных признаков можно попробовать создать признаки для каждого значения категориальных фичей (признак_категория_i) и присвоить ему количество данных категорий для каждого клиента. Также можно упорядочить статус кредита и подсчитать среднее значение (не стоит считать сумму, так как у всех клиентов разное количество строк и результаты будут искажаться)

#### CREDIT_ACTIVE - статус кредита

#### CREDIT_CURRENCY - валюта кредита

#### CREDIT_TYPE - тип кредита 


## Числовые признаки: 
### Агрегирование:
#### Для агрегирования числовых признаков можно создавать признаки с минимальным, средним и максимальным значение для каждого клиента. 

#### SK_ID_CURR - номер клиента

#### SK_ID_BUREAU - номер заявки

#### DAYS_CREDIT - количество прошедших дней с подачи заявки в отношении текущей заявки

#### CREDIT_DAY_OVERDUE - количество просроченных дней

#### DAYS_CREDIT_ENDDATE - количество дней, в течении которых кредит действителен 

#### DAYS_ENDDATE_FACT - количество дней с момента окончания срока кредита 

#### AMT_CREDIT_MAX_OVERDUE - максимальная задолженность по кредиту

#### CNT_CREDIT_PROLONG - сколько раз была продлена дата до погашения

#### AMT_CREDIT_SUM - сумма кредитов

#### AMT_CREDIT_SUM_DEBT - сумма задолженностей

#### AMT_CREDIT_SUM_LIMIT - текущий кредит

#### AMT_CREDIT_SUM_OVERDUE - текущий просроченный платеж по кредиту

#### DAYS_CREDIT_UPDATE - количество дней, когда было получено последнее обновление по кредиту

#### AMT_ANNUITY - аннуитет данных кредитных бюро. 

In [12]:
from treatment import ProcessingOmissions as pso

In [13]:
feature_str, feature_int = als.get_lists_type_feature(bureau, list(bureau))

In [14]:
from aggregation import AggregationData as agd

In [127]:
agr_str = aggregation_categorical_feature(bureau, feature_str)